### Preparing the Julia environment

In [1]:
cd(@__DIR__)
using Pkg
Pkg.activate(".")
Pkg.instantiate()

  Activating project at `~/projects/research/phylogeneticCausalAnalysis/code`


In [49]:
using CSV
using DataFrames
using Plots
using StatsPlots
using MCPhylo
using MCPhyloTree
plotlyjs()
using Distributions
using LinearAlgebra
using Pipe
using Statistics

In [3]:
import MCPhylo: SamplerTune, SymDistributionType, Sampler, ElementOrVector
import MCPhylo: sample!

include("prior_sampler.jl")

StatsBase.sample!

In [82]:
nChains = 2

2

### Loading the data

In [5]:
d = CSV.File("../data/data_cues.txt") |> DataFrame
show(d, allcols=true)

30×7 DataFrame
 Row │ Glottocode  Language    Genus              Case_Marking  Tight_Semantics  Rigid_Order  Verb_Middle 
     │ String15    String15    String31           Float64       Float64          Float64      Float64     
─────┼────────────────────────────────────────────────────────────────────────────────────────────────────
   1 │ stan1318    Arabic      Semitic                   0.036            0.218        0.655        0.391
   2 │ bulg1262    Bulgarian   Slavic                    0.028            0.144        0.782        0.966
   3 │ croa1245    Croatian    Slavic                    0.415            0.147        0.414        0.9
   4 │ czec1258    Czech       Slavic                    0.525            0.172        0.24         0.818
   5 │ dani1285    Danish      Germanic                  0.0              0.208        0.926        0.912
   6 │ dutc1256    Dutch       Germanic                  0.0              0.214        0.817        0.462
   7 │ stan1293    English    

In [63]:
corrplot(Array(d[:,4:end]))

In [76]:
cov(dArrayrrayrrayrray)

4×4 Matrix{Float64}:
  1.0        0.34742   -0.77153   -0.191945
  0.34742    1.0       -0.18423   -0.369734
 -0.77153   -0.18423    1.0        0.177435
 -0.191945  -0.369734   0.177435   1.0

### normalizing data



In [64]:
dataColumns = names(d)[4:end]


4-element Vector{String}:
 "Case_Marking"
 "Tight_Semantics"
 "Rigid_Order"
 "Verb_Middle"

In [65]:
normalize(x) = (x .- mean(x)) ./ std(x)

normalize (generic function with 1 method)

In [66]:
dArray = @pipe d |> 
    select(_, dataColumns) |>
    Array |>
    mapslices(normalize, _, dims=1);

In [67]:
corrplot(dArray)

### Loading the trees

In [68]:
trees = MCPhylo.ParseNewick("../data/posterior.tree");
nTrees = length(trees)
summary(trees)

"1000-element Vector{GeneralNode}"

### extracting covariance matrices from trees

In [69]:
sigmas = [MCPhyloTree.to_covariance(t) for t in trees];

# The model

- There are 30 data points
- $X$ is a $30\times (n+1)$ matrix with the first column being all-1. ($n$ is the number of independent variables.)
- $Y$ is a length-30 vector.
- $\beta$, the regression coefficient vector, is a vector of length $n+1$.
- $\rho$, the rate, is a positive real number.
- $\Sigma$ is a variance-covariance matrix (taken from a phylogenetic tree)


$$
\begin{aligned}
    Y &\sim \mathcal N(X\beta, \rho\Sigma)\\
    \beta_i &\sim \mathcal N(0, 10) &\forall i\\
    \sigma &\sim \mathrm{Exponential}(1)
\end{aligned}
$$

In [70]:
data = Dict{Symbol, Any}(
    :x => [ones(nrow(d)) dArray[:,1]], 
    :y => dArray[:,2],
    :sigmas => sigmas,
);

In [71]:
nCoefficients = size(data[:x],2)
inits = [
    Dict{Symbol, Any}(
        :idx => rand(1:nTrees),
        :x => data[:x],
        :y => data[:y],
        :sigmas => data[:sigmas],
        :β => rand(MvNormal(zeros(nCoefficients), 100)),
        :ρ => rand(Exponential()),
    )
    for i in 1:nChains
];

In [83]:
model = Model(
    y = Stochastic(1,
        (x,β,ρ,sigmas,idx) -> MvNormal(x*β, ρ*sigmas[Int(ceil(idx))]),
        false,
    ),
    β = Stochastic(1,
        () -> MvNormal(ones(nCoefficients),10),
        true,
    ),
    ρ = Stochastic(
        () -> Exponential(1),
        true,
    ),
    idx=Stochastic(
        () -> Uniform(0, nTrees),
        false,
    ),
);

In [113]:
scheme = [NUTS([:y, :β, :ρ]), Prior([:idx])];


In [114]:
setsamplers!(model, scheme);

In [115]:
sim = mcmc(
    model,
    data,
    inits,
    2000,
    burnin=1000,
    thin=1,
    chains=nChains,
    trees=false,
    verbose=true,
);


MCMC Simulation of 2000 Iterations x 2 Chains...


Chain 1: 100%|████████████████████████████| Time: 0:00:29 (14.96 ms/it)
2mChain 2: 100%|████████████████████████████| Time: 0:00:57 (28.67 ms/it)

In [116]:
gelmandiag(sim)

            PSRF 97.5%
      β[1] 1.030 1.126
      β[2] 1.238 1.791
         ρ 1.010 1.049
likelihood   NaN   NaN



In [117]:
plot(sim)

Press ENTER to draw next plot
stdin> 


2-element Vector{Plots.Plot}:
 Plot{Plots.PlotlyJSBackend() n=6}
 Plot{Plots.PlotlyJSBackend() n=6}

In [118]:
describe(sim)

Iterations = 1001:2000
Thinning interval = 1
Chains = 1,2
Samples per chain = 1000

Empirical Posterior Estimates:
              Mean        SD       Naive SE      MCSE       ESS   
      β[1] -0.4181874 10.6391575 0.237898793 1.455958536 53.396951
      β[2]  0.5772768  9.3420683 0.208894998 1.228281321 57.848271
         ρ  0.9636163  0.8894917 0.019889638 0.115493977 59.315075
likelihood         -∞        NaN         NaN         NaN       NaN

Quantiles:
               2.5%        25.0%       50.0%     75.0%      97.5%  
      β[1] -25.49740172 -7.95482984  2.6098541 6.2933879 17.1439553
      β[2] -19.68021200 -4.00532599 -0.5718129 7.4361678 17.1161062
         ρ   0.15149716  0.30121055  0.6913542 1.2805235  3.4503170
likelihood           -∞          -∞         -∞        -∞         -∞



In [119]:
hpd(sim)

             95% Lower   95% Upper
      β[1] -20.84283837 20.0009192
      β[2] -19.86673260 16.3562972
         ρ   0.13086295  2.7812612
likelihood           -∞         -∞

